In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL, DfToSqlMap
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import GrubbsTest

In [ ]:
# class OPCO:
#     r"""
#     A simple class to hold OPCO information
#     """
#     def __init__(
#         self, 
#         opco, 
#         name,
#         ID, 
#         states
#     ):
#         self.opco   = opco
#         self.name   = name
#         self.ID     = ID
#         self.states = states
        
        
# OPCO_ap = OPCO(
#     opco='ap', 
#     name='AEP-Appalachian Power', 
#     ID=1, 
#     states=['TN', 'VA', 'WV']
# )

# OPCO_im = OPCO(
#     opco='im', 
#     name='AEP-Indiana Michigan Power', 
#     ID=4, 
#     states=['IN', 'MI']
# )

# OPCO_oh = OPCO(
#     opco='oh', 
#     name='AEP-Ohio', 
#     ID=3, 
#     states=['IN', 'OH', 'OK']
# )

# OPCO_pso = OPCO(
#     opco='pso', 
#     name='AEP-Public Service Co of OK', 
#     ID=5, 
#     states=['OK']
# )

# OPCO_swp = OPCO(
#     opco='swp', 
#     name='AEP-Southwestern Electric Power', 
#     ID=6, 
#     states=['AR', 'LA', 'TX']
# )

# OPCO_tx = OPCO(
#     opco='tx', 
#     name=AEP-Texas, 
#     ID=7, 
#     states=['TX']
# )

In [ ]:
opco_to_id = dict(
    ap  = 1, 
    ky  = 2, 
    oh  = 3, 
    im  = 4, 
    pso = 5, 
    swp = 6, 
    tx  = 7
)

In [ ]:
save_dfs=True

date_0 = '2024-12-29'
date_1 = '2025-01-04'

# states=['OH']
states=None

# opcos='oh'
opcos=None

# CI_NB_min=15
CI_NB_min=None

conn_outages = Utilities.get_utldb01p_oracle_connection()
#--------------------------------------------------
# For this DAQ, I am doing one opco at a time
# assert(isinstance(opcos, str))
save_dir_base = r'C:\Users\s346557\Documents\LocalData\dovs_check'
save_subdir = f"{date_0.replace('-','')}_{date_1.replace('-','')}"
save_dir_base = os.path.join(save_dir_base, save_subdir)
if opcos:
    save_dir_base = os.path.join(save_dir_base, opcos)
else:
    save_dir_base = os.path.join(save_dir_base, 'AllOPCOs')
if not os.path.exists(save_dir_base):
    os.makedirs(save_dir_base)

In [ ]:
#--------------------------------------------------
sql_outage_full = DOVSOutages_SQL.build_sql_std_outage(
    mjr_mnr_cause=None, 
    include_premise=True, 
    date_range=[date_0, date_1], 
    states=states, 
    opcos=opcos, 
    CI_NB_min=CI_NB_min
).get_sql_statement()
#-----
print(f'sql_outage_full:\n{sql_outage_full}\n\n')
#-----
df_outage_OG = pd.read_sql_query(
    sql_outage_full, 
    conn_outages, 
    dtype={
        'CI_NB':np.int32, 
        'CMI_NB':np.float64, 
        'OUTG_REC_NB':np.int32
    }
)
#-----
print(f"df_outage_OG.shape = {df_outage_OG.shape}")
print(f"# OUTG_REC_NBs     = {df_outage_OG['OUTG_REC_NB'].nunique()}")

In [ ]:
# save_dir_base = r'C:\Users\s346557\Documents\LocalData\dovs_check\forAmanda\SN_882735506'
# states=['OH']
# opcos='oh'
# conn_outages = Utilities.get_utldb01p_oracle_connection()
# #--------------------------------------------------
# sql_outage_full = DOVSOutages_SQL.build_sql_std_outage(
#     mjr_mnr_cause=None, 
#     include_premise=True, 
#     outg_rec_nbs=['11145126', '12522511', '13149594', '13169176', '13219189'], 
#     states=states
# ).get_sql_statement()
# #-----
# print(f'sql_outage_full:\n{sql_outage_full}\n\n')
# #-----
# df_outage_OG = pd.read_sql_query(
#     sql_outage_full, 
#     conn_outages, 
#     dtype={
#         'CI_NB':np.int32, 
#         'CMI_NB':np.float64, 
#         'OUTG_REC_NB':np.int32
#     }
# )
# #-----
# print(f"df_outage_OG.shape = {df_outage_OG.shape}")
# print(f"# OUTG_REC_NBs     = {df_outage_OG['OUTG_REC_NB'].nunique()}")

In [ ]:
#-------------------------
if save_dfs:
    df_outage_OG.to_pickle(os.path.join(save_dir_base, 'df_outage.pkl'))
#-------------------------
outg_rec_nbs_all = df_outage_OG['OUTG_REC_NB'].unique().tolist()
df_outage = df_outage_OG.copy()

In [ ]:
start=time.time()
#-----
# df_mp_outg_OG = build_active_MP_for_xfmrs_in_outages_df(
#     df_outage=df_outage, 
#     prem_nb_col='PREMISE_NB', 
#     is_slim=False, 
#     addtnl_mp_df_curr_cols=['technology_tx'], 
#     addtnl_mp_df_hist_cols=['technology_tx']
# )
# #-----
# print(f'Time for build_active_MP_for_xfmrs_in_outages_df: {time.time()-start}')
df_mp_outg_OG = build_active_MP_for_outages_df(
    df_outage=df_outage, 
    prem_nb_col='PREMISE_NB', 
    is_slim=False, 
    addtnl_mp_df_curr_cols=['technology_tx'], 
    addtnl_mp_df_hist_cols=['technology_tx'], 
    assert_all_PNs_found=False
)
#-----
print(f'Time for build_active_MP_for_outages_df: {time.time()-start}')
start=time.time()
#-----
df_mp_outg_OG['inst_ts'] = pd.to_datetime(df_mp_outg_OG['inst_ts'])
df_mp_outg_OG['rmvl_ts'] = pd.to_datetime(df_mp_outg_OG['rmvl_ts'])
#-------------------------
if save_dfs:
    df_mp_outg_OG.to_pickle(os.path.join(save_dir_base, 'df_mp_outg_b4_dupl_rmvl.pkl'))
#-------------------------
df_mp_outg = MeterPremise.drop_approx_mp_duplicates(
    mp_df = df_mp_outg_OG.copy(), 
    fuzziness=pd.Timedelta('1 hour'), 
    assert_single_overlap=True, 
    addtnl_groupby_cols=['OUTG_REC_NB', 'technology_tx'], 
    gpby_dropna=False
)
#-----
print(f'Time for drop_approx_mp_duplicates: {time.time()-start}')
#-------------------------
if save_dfs:
    df_mp_outg.to_pickle(os.path.join(save_dir_base, 'df_mp_outg.pkl'))
#-------------------------

In [ ]:
# Some premises are listed in DOVS are simply not found in AMI
print(f"#PNs DOVS: {df_outage['PREMISE_NB'].nunique()}")
print(f"#PNs AMI:  {df_mp_outg['prem_nb'].nunique()}")

In [ ]:
# Really only want one entry per meter (here, meter being a mfr_devc_ser_nbr/prem_nb combination)
# ALthough drop_duplicates was used, multiple entries could still exist if, e.g., a meter has two
#   non-fuzzy-overlapping intervals
assert(all(df_mp_outg[['mfr_devc_ser_nbr', 'prem_nb', 'OUTG_REC_NB']].value_counts()==1))

# # Simple-minded (if assertion fails): Let's just keep the one with the most recent install date
# df_mp_outg = df_mp_outg.iloc[df_mp_outg.reset_index().groupby(['mfr_devc_ser_nbr', 'prem_nb', 'OUTG_REC_NB'])['inst_ts'].idxmax()]
# assert(all(df_mp_outg[['mfr_devc_ser_nbr', 'prem_nb', 'OUTG_REC_NB']].value_counts()==1))

In [ ]:
# # Keep only trsf_pole_nbs with all meters of type AMI
# df_mp_outg_ami = df_mp_outg.groupby(['trsf_pole_nb']).filter(lambda x: all(x['technology_tx']=='AMI'))

# Keep only outages with all meters of typr AMI
df_mp_outg_ami = df_mp_outg.groupby(['OUTG_REC_NB']).filter(lambda x: all(x['technology_tx']=='AMI'))
#-------------------------
if save_dfs:
    df_mp_outg_ami.to_pickle(os.path.join(save_dir_base, 'df_mp_outg_ami.pkl'))
#-------------------------

In [ ]:
print(df_mp_outg.shape)
print(df_mp_outg_ami.shape)

In [ ]:
print(df_mp_outg['OUTG_REC_NB'].nunique())
print(df_mp_outg_ami['OUTG_REC_NB'].nunique())

In [ ]:
ami_outg_rec_nbs = df_mp_outg_ami['OUTG_REC_NB'].unique().tolist()
df_outage_ami = df_outage[df_outage['OUTG_REC_NB'].isin(ami_outg_rec_nbs)]

# NOTE:
- Here, when running batch DAQ, we need to merge DOVS with MP so we can group by trsf_pole_nb later when actually running the DAQ
- This is mainly a convenience thing, so that (along with OUTG_REC_NB and operating company info) are ready in the output DAQ files (otherwise, I would have to populate these data at run time, which would be annoying)

In [ ]:
df_outage_ami_w_mp = DOVSOutages.merge_df_outage_with_mp(
    df_outage=df_outage_ami.copy(), 
    df_mp=df_mp_outg_ami, 
    merge_on_outg=['OUTG_REC_NB', 'PREMISE_NB'], 
    merge_on_mp=['OUTG_REC_NB', 'prem_nb'], 
    cols_to_include_mp=None, 
    drop_cols = None, 
    rename_cols=None, 
    inplace=True
)
#-------------------------
if save_dfs:
    df_outage_ami_w_mp.to_pickle(os.path.join(save_dir_base, 'df_outage_ami_w_mp.pkl'))
#-------------------------

In [ ]:
df_outage_ami_w_mp['OUTG_REC_NB'].nunique()

In [ ]:
df_outage_ami_w_mp_slim = DOVSOutages.consolidate_df_outage(
    df_outage_ami_w_mp, 
    addtnl_grpby_cols=['trsf_pole_nb'], 
    set_outg_rec_nb_as_index=False, 
    gpby_dropna=False
)

In [ ]:
df_outage_ami_w_mp_slim = DOVSOutages.set_search_time_in_outage_df(
    df_outage=df_outage_ami_w_mp_slim, 
    search_time_half_window=pd.Timedelta('24 hours')
)
#-------------------------
if save_dfs:
    df_outage_ami_w_mp_slim.to_pickle(os.path.join(save_dir_base, 'df_outage_ami_w_mp_slim.pkl'))
#-------------------------

In [ ]:
df_outage_ami_w_mp_slim['OUTG_REC_NB'].nunique()

In [ ]:
# The following will find OUTG_REC_NB, trsf_pole_nb groups (since DOVSOutages.consolidate_df_outage was run with 
#   addtnl_grpby_cols=['trsf_pole_nb']) for which the PNs found by DOVS are not equal to those found in MP
# BUT, if they're missing from MP, does this necessarily mean they'll be missing from interval data?!
#   SO, I'm not 100% sure whether or not these should be eliminated at this stage.
#   Keeping them at this point will just lead to a slightly longer data collection time.
#   If they should be left out, the functionality of check_DOVS will eliminate them
df_outg_missing_PNs = df_outage_ami_w_mp_slim[df_outage_ami_w_mp_slim.apply(
    lambda x: len(set(x['PREMISE_NBS']).symmetric_difference(set(x['prem_nb'])))!=0, 
    axis=1
)]
outg_rec_nbs_to_exclude = df_outg_missing_PNs['OUTG_REC_NB'].unique().tolist()
#-----
df_exclude_missing = df_outage_ami_w_mp_slim[~df_outage_ami_w_mp_slim['OUTG_REC_NB'].isin(outg_rec_nbs_to_exclude)]
#-----
print('With excluding outages with premises missing from MP')
print(f"# Outages: {df_exclude_missing['OUTG_REC_NB'].nunique()}")
print(f"#PNs DOVS: {len(set(df_exclude_missing['PREMISE_NBS'].sum()))}")
print(f"#PNs AMI:  {len(set(df_exclude_missing['prem_nb'].sum()))}")
print()
#-----
print('Without excluding outages with premises missing from MP')
print(f"# Outages: {df_outage_ami_w_mp_slim['OUTG_REC_NB'].nunique()}")
print(f"#PNs DOVS: {len(set(df_outage_ami_w_mp_slim['PREMISE_NBS'].sum()))}")
print(f"#PNs AMI:  {len(set(df_outage_ami_w_mp_slim['prem_nb'].sum()))}")

## AMI NonVee

In [ ]:
# df_outage_ami_w_mp_slim = pd.read_pickle(os.path.join(save_dir_base, 'df_outage_ami_w_mp_slim.pkl'))

In [ ]:
#-------------------------
df_construct_type=DFConstructType.kRunSqlQuery
contstruct_df_args_ami=None
# addtnl_groupby_cols=['OUTG_REC_NB', 'trsf_pole_nb']
addtnl_groupby_cols=['OUTG_REC_NB', 'trsf_pole_nb', 'OPERATING_UNIT_ID']

cols_of_interest_ami = TableInfos.AMINonVee_TI.std_columns_of_interest
# batch_size=100
batch_size=25
verbose=True
n_update=1

In [ ]:
ami_sql_function_kwargs = dict(
    cols_of_interest=cols_of_interest_ami, 
    df_outage=df_outage_ami_w_mp_slim, 
    join_mp_args=False, 
    df_args = dict(
        addtnl_groupby_cols=addtnl_groupby_cols, 
        mapping_to_ami=DfToSqlMap(df_col='PREMISE_NBS', kwarg='premise_nbs', sql_col='aep_premise_nb'), 
        is_df_consolidated=True
    ), 
    field_to_split='df_outage', 
    field_to_split_location_in_kwargs=['df_outage'], 
    save_and_dump=True,  
    sort_coll_to_split=True,
    batch_size=batch_size, verbose=verbose, n_update=n_update
)
addtnl_ami_sql_function_kwargs = dict(
    build_sql_function_kwargs=dict(opco=opcos)
)
ami_sql_function_kwargs = {**ami_sql_function_kwargs, 
                           **addtnl_ami_sql_function_kwargs}


save_args = dict(
    save_to_file=True, 
    save_dir = os.path.join(save_dir_base, 'AMINonVee'), 
    save_name=r'ami_nonvee.csv', 
    index=True
)

In [ ]:
# start=time.time()
# ami_nonvee = AMINonVee(
#     df_construct_type=df_construct_type, 
#     contstruct_df_args = contstruct_df_args_ami, 
#     build_sql_function=AMINonVee_SQL.build_sql_usg_for_outages, 
#     build_sql_function_kwargs=ami_sql_function_kwargs, 
#     init_df_in_constructor=True, 
#     save_args=save_args
# )
# build_time = time.time()-start
# print(build_time)

In [ ]:
# start=time.time()

# while True:
#     try:
#         ami_nonvee = AMINonVee(
#             df_construct_type=df_construct_type, 
#             contstruct_df_args = contstruct_df_args_ami, 
#             build_sql_function=AMINonVee_SQL.build_sql_usg_for_outages, 
#             build_sql_function_kwargs=ami_sql_function_kwargs, 
#             init_df_in_constructor=True, 
#             save_args=save_args
#         )
#         break # stop the loop if the function completes sucessfully
#     except Exception as e:
#         print("Function errored out!", e)
#         print("Retrying ... ")
        
# build_time = time.time()-start
# print(build_time)

In [ ]:
start=time.time()

exit_status = Utilities.run_tryexceptwhile_process(
    func                = AMINonVee,
    func_args_dict      = dict(
        df_construct_type         =df_construct_type, 
        contstruct_df_args        = contstruct_df_args_ami, 
        build_sql_function        = AMINonVee_SQL.build_sql_usg_for_outages, 
        build_sql_function_kwargs = ami_sql_function_kwargs, 
        init_df_in_constructor    = True, 
        save_args                 = save_args
    ), 
    max_calls_per_min   = 1, 
    lookback_period_min = 15, 
    max_calls_absolute  = 1000, 
    verbose             = True
)
print(f'exit_status = {exit_status}')

build_time = time.time()-start
print(build_time)

# AMI End Events

In [ ]:
#-------------------------
df_construct_type=DFConstructType.kRunSqlQuery
contstruct_df_args_end_events=None
# addtnl_groupby_cols=['OUTG_REC_NB', 'trsf_pole_nb']
addtnl_groupby_cols=['OUTG_REC_NB', 'trsf_pole_nb', 'OPERATING_UNIT_ID']

cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
batch_size=100
verbose=True
n_update=1

In [ ]:
end_events_sql_function_kwargs = dict(
    cols_of_interest=cols_of_interest_end_dev_event, 
    df_outage=df_outage_ami_w_mp_slim, 
    join_mp_args=False, 
    df_args = dict(
        addtnl_groupby_cols=addtnl_groupby_cols, 
        mapping_to_ami=DfToSqlMap(df_col='PREMISE_NBS', kwarg='premise_nbs', sql_col='aep_premise_nb'), 
        is_df_consolidated=True
    ), 
    field_to_split='df_outage', 
    field_to_split_location_in_kwargs=['df_outage'], 
    save_and_dump=True, 
    sort_coll_to_split=True,
    batch_size=batch_size, verbose=verbose, n_update=n_update
)
addtnl_end_events_sql_function_kwargs = dict(
    build_sql_function_kwargs=dict(opco=opcos)
)
end_events_sql_function_kwargs = {**end_events_sql_function_kwargs, 
                                  **addtnl_end_events_sql_function_kwargs}

end_events_save_args = dict(
    save_to_file=True, 
    save_dir = os.path.join(save_dir_base, 'EndEvents'), 
    save_name=r'end_events.csv', 
    index=True
)

In [ ]:
# start=time.time()
# end_events = AMIEndEvents(
#     df_construct_type=df_construct_type, 
#     contstruct_df_args = contstruct_df_args_end_events, 
#     build_sql_function=AMIEndEvents_SQL.build_sql_end_events_for_outages, 
#     build_sql_function_kwargs=end_events_sql_function_kwargs, 
#     init_df_in_constructor=True, 
#     save_args=end_events_save_args
# )
# end_events_build_time = time.time()-start

In [ ]:
# start=time.time()

# while True:
#     try:
#         end_events = AMIEndEvents(
#             df_construct_type=df_construct_type, 
#             contstruct_df_args = contstruct_df_args_end_events, 
#             build_sql_function=AMIEndEvents_SQL.build_sql_end_events_for_outages, 
#             build_sql_function_kwargs=end_events_sql_function_kwargs, 
#             init_df_in_constructor=True, 
#             save_args=end_events_save_args
#         )
#         break # stop the loop if the function completes sucessfully
#     except Exception as e:
#         print("Function errored out!", e)
#         print("Retrying ... ")
        
# end_events_build_time = time.time()-start
# print(end_events_build_time)

In [ ]:
start=time.time()

exit_status = Utilities.run_tryexceptwhile_process(
    func                = AMIEndEvents,
    func_args_dict      = dict(
        df_construct_type         = df_construct_type, 
        contstruct_df_args        = contstruct_df_args_end_events, 
        build_sql_function        = AMIEndEvents_SQL.build_sql_end_events_for_outages, 
        build_sql_function_kwargs = end_events_sql_function_kwargs, 
        init_df_in_constructor    = True, 
        save_args                 = end_events_save_args
    ), 
    max_calls_per_min   = 1, 
    lookback_period_min = 15, 
    max_calls_absolute  = 1000, 
    verbose             = True
)
print(f'exit_status = {exit_status}')

build_time = time.time()-start
print(build_time)

In [ ]:
assert(0)

In [ ]:
len(set(df_outage_ami_w_mp_slim[df_outage_ami_w_mp_slim['OUTG_REC_NB']==11145126]['PREMISE_NBS'].sum()))

In [ ]:
df_outage_ami_w_mp[df_outage_ami_w_mp['OUTG_REC_NB']==11145126]['PREMISE_NB'].nunique()

In [ ]:
pns_old = df_outage_ami_w_mp_slim_OLD[df_outage_ami_w_mp_slim_OLD['OUTG_REC_NB']==11145126]['PREMISE_NBS']

In [ ]:
len(set(pns_old.values[0]))

In [ ]:
PNs = df_outage_ami_w_mp[df_outage_ami_w_mp['OUTG_REC_NB']==11145126]['PREMISE_NB'].unique().tolist()
pns = df_outage_ami_w_mp[df_outage_ami_w_mp['OUTG_REC_NB']==11145126]['prem_nb'].unique().tolist()

In [ ]:
set(pns).difference(set(PNs))

In [ ]:
set(PNs).difference(set(pns))

In [ ]:
huh = df_outage_ami_w_mp[df_outage_ami_w_mp['OUTG_REC_NB']==11145126].copy()

In [ ]:
huh[huh['PREMISE_NB'].isin(list(set(PNs).difference(set(pns))))]

In [ ]:
huh[huh['PREMISE_NB'].isin(list(set(PNs).difference(set(pns))))]['OUTG_REC_NB'].unique()

In [ ]:
df_mp_outg_OG[df_mp_outg_OG['prem_nb']=='077879337']

In [ ]:
df_outage

In [ ]:
df_outage_ami_w_mp = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_check\Mico2\AllOPCOs\df_outage_ami_w_mp.pkl')

In [ ]:
df_outage_ami_w_mp